<a href="https://colab.research.google.com/github/karthikmohan1702/EVA5/blob/master/EVA4_Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
# Accuracy 99.46(0.03) Params - 12,026
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3)# IP=28x28, OP=26x26, RF=3x3 
        self.conv2 = nn.Conv2d(16, 16, 3)# IP=26x26, OP=24x24, RF=5x5 
        self.batchnorm2d_1 = nn.BatchNorm2d(16)
        self.dropout_1 = nn.Dropout(0.25)

        self.conv3 = nn.Conv2d(16, 16, 3)# IP=24x24, OP=22x22, RF=7x7 
        self.batchnorm2d_2 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2)# IP=22x22, OP=11x11, RF=14x14 
        self.dropout_2 = nn.Dropout(0.15)

        self.conv5 = nn.Conv2d(16, 16, 3)# IP=11x11, OP=9x9, RF=16x16 
        self.batchnorm2d_4 = nn.BatchNorm2d(16)
        

        self.conv7 = nn.Conv2d(16, 16, 3)# IP=9x9, OP=7x7, RF=18x18 
        self.conv8 = nn.Conv2d(16, 16, 3)# IP=7x7, OP=5x5, RF=20x20 
        self.pool2 = nn.AvgPool2d(5)
        self.fc = nn.Linear(16, 10)
    def forward(self, x):
        x = self.dropout_1(self.batchnorm2d_1(F.relu(self.conv2(F.relu(self.conv1(x))))))
        x = self.dropout_2(self.pool1(self.batchnorm2d_2(F.relu(self.conv3(x)))))
  
        x = self.batchnorm2d_4(F.relu(self.conv5(x)))
        x = F.relu(self.conv7(x))
        x = F.relu(self.conv8(x))
        x = self.pool2(x)        
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        
        return F.log_softmax(x,dim=1)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
            Conv2d-2           [-1, 16, 24, 24]           2,320
       BatchNorm2d-3           [-1, 16, 24, 24]              32
           Dropout-4           [-1, 16, 24, 24]               0
            Conv2d-5           [-1, 16, 22, 22]           2,320
       BatchNorm2d-6           [-1, 16, 22, 22]              32
         MaxPool2d-7           [-1, 16, 11, 11]               0
           Dropout-8           [-1, 16, 11, 11]               0
            Conv2d-9             [-1, 16, 9, 9]           2,320
      BatchNorm2d-10             [-1, 16, 9, 9]              32
           Conv2d-11             [-1, 16, 7, 7]           2,320
           Conv2d-12             [-1, 16, 5, 5]           2,320
        AvgPool2d-13             [-1, 16, 1, 1]               0
           Linear-14                   

In [4]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=False, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...



Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.03, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0845, Accuracy: 9734/10000 (97.34%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0487, Accuracy: 9852/10000 (98.52%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0266, Accuracy: 9925/10000 (99.25%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0223, Accuracy: 9934/10000 (99.34%)



loss=0.0038175806403160095 batch_id=468: 100%|██████████| 469/469 [00:14<00:00, 33.40it/s]



Test set: Average loss: 0.0179, Accuracy: 9941/10000 (99.41%)

